In [45]:
import numpy as np
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import dblquad
from scipy.integrate import quad

from scipy.optimize import approx_fprime
from mpl_toolkits.mplot3d import Axes3D
from sympy import *

# np.tensordot
# np.cross
# np.dot

In [3]:
def vt(X: np.ndarray) -> np.ndarray:
    """Returns the tangent and azimutal unit vectors at point X on the unit sphere.
    Args:
        X (np.ndarray): A 3D point on the unit sphere.
        Returns:
        [-y/(x^2+y^2)^0.5, x/(x^2+y^2)^0.5, 0]"""
    s = np.sqrt(X[0]**2 + X[1]**2)
    return np.array([-X[1]/s, X[0]/s, 0])

def vr(X: np.ndarray) -> np.ndarray:
    """Returns the radial unit vector at point X on the unit sphere.
    Args:
        X (np.ndarray): A 3D point on the unit sphere.
        Returns:
        [X[0], X[1], X[2]]/(np.sqrt(X[0]**2 + X[1]**2 + X[2]**2))"""
    s = np.sqrt(X[0]**2 + X[1]**2+ X[2]**2)
    return np.array([X[0], X[1], X[2]])/s

def th(X: np.ndarray):
    """Returns the polar angle theta at point X on the unit sphere theta."""
    return np.arccos(X[2]/np.sqrt(X[0]**2 + X[1]**2 + X[2]**2))

def ncone(X: np.ndarray, w) -> np.ndarray:
    """Returns the director n on the boundary of the cone at point X on the unit sphere.
    Args:
        X (np.ndarray): A 3D point on the unit sphere.
        thetacone (float): The half-angle of the cone in radians.
        w (int): The winding number of the director along the boundary on the cone.
        Returns:
        n = cos(w*phi)*vt + sin(w*phi)*vr
        where phi = arctan2(y,x)
        
        if x=0 then we separate fo different cases"""
    if X[0] == 0 and X[1] == 0:
        return np.zeros(3)
    elif X[0] == 0 and X[1] > 0:   
        return np.cos(w * np.pi / 2) * vt(X) + np.sin(w * np.pi / 2) * vr(X)
    elif X[0] == 0 and X[1] < 0:
        return np.cos(w * np.pi / 2) * vt(X) - np.sin(w * np.pi / 2) * vr(X)        
    else:
        return np.cos(w * np.arctan2(X[1], X[0])) * vt(X) + np.sin(w * np.arctan2(X[1], X[0])) * vr(X)
    


def nhat(X, thetacone: float, w) -> np.ndarray:
    """Returns the director n at point X on the unit sphere.
    Args:
        X (np.ndarray): A 3D point on the unit sphere.
        thetacone (float): The half-angle of the cone in radians.
        w (int): The winding number of the director along the boundary on the cone.
        Returns:
        n = """
    n = th(X) * ncone(X, w) + (thetacone - th(X)) * np.array([0, 0, 1])
    return n / np.linalg.norm(n)


In [4]:
def epsilon3d():
    eps = np.zeros((3,3,3))
    eps[0,1,2]=eps[1,2,0]=eps[2,0,1]=1
    eps[0,2,1]=eps[1,0,2]=eps[2,1,0]=-1
    return eps

In [5]:
def integrand(point, angle, winding) -> np.ndarray:
    """epsilon_ijk epsilon_pqr n_q,i n_r,k n_p at point on the unit sphere.
    Args:
        point (np.ndarray): A 3D point on the unit sphere.
        angle (float): The half-angle of the cone in radians.
        winding (int): The winding number of the director along the boundary on the cone.
        Returns:
        A scalar representing the integrand at the given point."""
    grad_nx = approx_fprime(point, lambda p: nhat(p, angle, winding)[0], epsilon=1e-8)  # x-component - 3d vector
    grad_ny = approx_fprime(point, lambda p: nhat(p, angle, winding)[1], epsilon=1e-8)  # y-component - 3d vector
    grad_nz = approx_fprime(point, lambda p: nhat(p, angle, winding)[2], epsilon=1e-8)  # z-component - 3d vector

    n = nhat(point, angle, winding)  # 3d vector
    dn = np.stack((grad_nx, grad_ny, grad_nz), axis=0)  # 3x3 matrix
    r = np.array([point[0], point[1], point[2]])

    return np.round(0.5 * np.einsum('ijk,pqr,qj,rk,p,i', epsilon3d(), epsilon3d(), dn, dn, n, r) * th(point), decimals=8) # scalar: epsilon_ijk epsilon_pqr n_q,i n_r,k n_p


In [60]:
from sympy import latex

x, y, z , a, b, c, d, e, f, g, h, i, j, k, l = symbols('x y z a b c d e f g h i j k l')
nx_x = symbols(r'\partial_{x}{n_{x}}')
nx_y = symbols(r'\partial_{y}{n_{x}}')
nx_z = symbols(r'\partial_{z}{n_{x}}')
ny_x = symbols(r'\partial_{x}{n_{y}}')
ny_y = symbols(r'\partial_{y}{n_{y}}')
ny_z = symbols(r'\partial_{z}{n_{y}}')
nz_x = symbols(r'\partial_{x}{n_{z}}')
nz_y = symbols(r'\partial_{y}{n_{z}}')
nz_z = symbols(r'\partial_{z}{n_{z}}')
r = [x,y,z]
r
nx = symbols(r'n_{x}')
ny = symbols(r'n_{y}')
nz = symbols(r'n_{z}')
n = [nx, ny, nz] 
n
dn = Matrix([[nx_x, nx_y, nx_z],
             [ny_x, ny_y, ny_z],
             [nz_x, nz_y, nz_z]])



I = np.einsum('ijk,pqr,qj,rk,p,i', epsilon3d(), epsilon3d(), dn, dn, n, r)/2
latex_expr = latex(I)
print(latex_expr)

1.0 \partial_{x}{n_{x}} \partial_{y}{n_{y}} n_{z} z - 1.0 \partial_{x}{n_{x}} \partial_{y}{n_{z}} n_{y} z - 1.0 \partial_{x}{n_{x}} \partial_{z}{n_{y}} n_{z} y + 1.0 \partial_{x}{n_{x}} \partial_{z}{n_{z}} n_{y} y - 1.0 \partial_{x}{n_{y}} \partial_{y}{n_{x}} n_{z} z + 1.0 \partial_{x}{n_{y}} \partial_{y}{n_{z}} n_{x} z + 1.0 \partial_{x}{n_{y}} \partial_{z}{n_{x}} n_{z} y - 1.0 \partial_{x}{n_{y}} \partial_{z}{n_{z}} n_{x} y + 1.0 \partial_{x}{n_{z}} \partial_{y}{n_{x}} n_{y} z - 1.0 \partial_{x}{n_{z}} \partial_{y}{n_{y}} n_{x} z - 1.0 \partial_{x}{n_{z}} \partial_{z}{n_{x}} n_{y} y + 1.0 \partial_{x}{n_{z}} \partial_{z}{n_{y}} n_{x} y + 1.0 \partial_{y}{n_{x}} \partial_{z}{n_{y}} n_{z} x - 1.0 \partial_{y}{n_{x}} \partial_{z}{n_{z}} n_{y} x - 1.0 \partial_{y}{n_{y}} \partial_{z}{n_{x}} n_{z} x + 1.0 \partial_{y}{n_{y}} \partial_{z}{n_{z}} n_{x} x + 1.0 \partial_{y}{n_{z}} \partial_{z}{n_{x}} n_{y} x - 1.0 \partial_{y}{n_{z}} \partial_{z}{n_{y}} n_{x} x


In [62]:
integrand(np.array([0,1,0]), np.pi/4, 1)

1e-08

In [6]:
# Implementation: Convert your integrand to spherical coordinates

def spherical_to_cartesian(theta, phi, r=1.0):
    """Convert spherical coordinates to Cartesian"""
    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)  
    z = r * np.cos(theta)
    return np.array([x, y, z])

def integrand_spherical(theta, phi, angle=np.pi/4, winding=1):
    """
    Your integrand function adapted for spherical coordinates
    
    This function:
    1. Converts (θ,φ) to Cartesian point on unit sphere
    2. Calls your existing integrand function
    3. Returns scalar value ready for integration
    """
    # Convert to Cartesian coordinates
    point_cartesian = spherical_to_cartesian(theta, phi)
    
    # Call your existing integrand function
    result = integrand(point_cartesian, angle, winding)
    
    return result

In [ ]:
# Method 1: Using scipy.integrate.dblquad (RECOMMENDED)

def charge(angle=np.pi/2, winding=0):
    """
    Compute surface integral using scipy.integrate.dblquad
    
    ∫∫_sphere integrand(θ,φ) sin(θ) dθ dφ
    """
    
    def integrand_with_jacobian(theta, phi):
        """Include the sin(θ) Jacobian factor"""
        return integrand_spherical(theta, phi, angle, winding) * np.sin(theta)
    
    """ Integrate over the cap: θ ∈ [0,angle], φ ∈ [0,2π] """
    result, error = dblquad(
        integrand_with_jacobian,
        0, 2*np.pi,          # φ limits
        lambda phi: 10 ** -8,       # θ lower bound (constant)
        lambda phi: angle    # θ upper bound (constant)
    )

    return np.round(result / (4 * np.pi), 3), np.round(error / (4 * np.pi), 2)



In [43]:
q, err =  charge(np.pi/100)
q

0.01